# Generate Categories Data

Generate INSERT SQL statements cho bảng categories và lưu vào file `category.insert.sql`

---



## 1. Import Libraries

In [7]:
import os
import json
import random
import re
import unicodedata
import psycopg2
from datetime import datetime
from dotenv import load_dotenv

load_dotenv()

SUPABASE_CONNECTION_STRING = os.getenv("SUPABASE_CONNECTION_STRING")

print("✅ Đã import tất cả thư viện")

✅ Đã import tất cả thư viện


## 2. Helper Funcions

In [2]:
def slugify(text):
    """Convert Vietnamese text to URL-friendly slug"""
    # Remove leading/trailing spaces and lowercase
    text = text.strip().lower()
    
    # Vietnamese character mapping
    from_chars = "àáạảãâầấậẩẫăằắặẳẵèéẹẻẽêềếệểễìíịỉĩòóọỏõôồốộổỗơờớợởỡùúụủũưừứựửữỳýỵỷỹđ·/_,:;"
    to_chars = "aaaaaaaaaaaaaaaaaeeeeeeeeeeeiiiiiooooooooooooooooouuuuuuuuuuuyyyyyd------"
    
    # Replace Vietnamese characters
    for i in range(len(from_chars)):
        text = text.replace(from_chars[i], to_chars[i])
    
    # Remove non-alphanumeric characters except spaces and hyphens
    text = re.sub(r'[^a-z0-9 -]', '', text)
    
    # Replace spaces with hyphens
    text = re.sub(r'\s+', '-', text)
    
    # Replace multiple hyphens with single hyphen
    text = re.sub(r'-+', '-', text)
    
    return text.strip('-')

print("✅ Đã định nghĩa các hàm hỗ trợ: slugify()")

✅ Đã định nghĩa các hàm hỗ trợ: slugify()


## 3. Main Execution

In [3]:
'''
Cấu trúc bảng categories:
create table public.categories (
  id serial not null,
  name character varying(50) not null,
  parent_id integer null,
  status character varying(10) not null default 'active'::character varying,
  deleted boolean not null default false,
  description character varying(1000) null,
  created_by integer null,
  updated_by integer null,
  created_at timestamp without time zone not null default now(),
  updated_at timestamp without time zone not null default now(),
  slug character varying(100) null,
  cat_image text null,
  constraint categories_pkey primary key (id),
  constraint categories_slug_key unique (slug),
  constraint fk_categories_parent foreign KEY (parent_id) references categories (id)
) TABLESPACE pg_default;
'''

"\nCấu trúc bảng categories:\ncreate table public.categories (\n  id serial not null,\n  name character varying(50) not null,\n  parent_id integer null,\n  status character varying(10) not null default 'active'::character varying,\n  deleted boolean not null default false,\n  description character varying(1000) null,\n  created_by integer null,\n  updated_by integer null,\n  created_at timestamp without time zone not null default now(),\n  updated_at timestamp without time zone not null default now(),\n  slug character varying(100) null,\n  cat_image text null,\n  constraint categories_pkey primary key (id),\n  constraint categories_slug_key unique (slug),\n  constraint fk_categories_parent foreign KEY (parent_id) references categories (id)\n) TABLESPACE pg_default;\n"

In [4]:
'''
Big categories and their subcategories:
1. Thời Trang
    - Thời Trang Name
    - Thời Trang Nữ 
    - Giày Dép
2. Điện Tử
    - Điện Thoại & Máy Tính Bảng
    - Laptop & PC 
    - Phụ Kiện Số
    - Máy Ảnh & Quay Phim
3. Nghệ Thuật & Sưu Tầm
    - Tranh Ảnh
    - Tượng & Điêu Khắc
    - Đồ Cổ & Hiếm
'''


'\nBig categories and their subcategories:\n1. Thời Trang\n    - Thời Trang Name\n    - Thời Trang Nữ \n    - Giày Dép\n2. Điện Tử\n    - Điện Thoại & Máy Tính Bảng\n    - Laptop & PC \n    - Phụ Kiện Số\n    - Máy Ảnh & Quay Phim\n3. Nghệ Thuật & Sưu Tầm\n    - Tranh Ảnh\n    - Tượng & Điêu Khắc\n    - Đồ Cổ & Hiếm\n'

In [ ]:
# Define category structure
categories = [
    {"name": "Thời Trang", "parent": None, "subcategories": [
        "Thời Trang Nam",
        "Thời Trang Nữ",
        "Giày Dép"
    ]},
    {"name": "Điện Tử", "parent": None, "subcategories": [
        "Điện Thoại & Máy Tính Bảng",
        "Laptop & PC",
        "Phụ Kiện Số",
        "Máy Ảnh & Quay Phim"
    ]},
    {"name": "Nghệ Thuật & Trang Sức", "parent": None, "subcategories": [
        "Tranh Ảnh",
        "Tượng & Điêu Khắc",
        "Trang Sức"
    ]}
]

# Connect to database
conn = psycopg2.connect(SUPABASE_CONNECTION_STRING)
cursor = conn.cursor()

try:
    # Insert parent categories and collect their IDs
    parent_ids = {}
    
    for category in categories:
        parent_name = category["name"]
        parent_slug = slugify(parent_name)
        
        cursor.execute(
            """
            INSERT INTO categories (name, slug, parent_id, status, deleted)
            VALUES (%s, %s, NULL, 'active', false)
            RETURNING id
            """,
            (parent_name, parent_slug)
        )
        parent_id = cursor.fetchone()[0]
        parent_ids[parent_name] = parent_id
        print(f"Inserted parent category: {parent_name} (ID: {parent_id})")
    
    # Insert subcategories
    for category in categories:
        parent_name = category["name"]
        parent_id = parent_ids[parent_name]
        
        for subcategory_name in category["subcategories"]:
            subcategory_slug = slugify(subcategory_name)
            
            cursor.execute(
                """
                INSERT INTO categories (name, slug, parent_id, status, deleted)
                VALUES (%s, %s, %s, 'active', false)
                RETURNING id
                """,
                (subcategory_name, subcategory_slug, parent_id)
            )
            subcategory_id = cursor.fetchone()[0]
            print(f"Inserted subcategory: {subcategory_name} (ID: {subcategory_id}) under {parent_name}")
    
    # Commit transaction
    conn.commit()
    print("\nSuccessfully inserted all categories")
    
except Exception as e:
    conn.rollback()
    print(f"Error: {e}")
    
finally:
    cursor.close()
    conn.close()

Error: duplicate key value violates unique constraint "categories_slug_key"
DETAIL:  Key (slug)=(thoi-trang) already exists.



In [8]:
# Update all existing category slugs with new slugify function
conn = psycopg2.connect(SUPABASE_CONNECTION_STRING)
cursor = conn.cursor()

try:
    # Fetch all categories
    cursor.execute("SELECT id, name FROM categories ORDER BY id")
    categories_to_update = cursor.fetchall()
    
    # Update each category slug
    for cat_id, cat_name in categories_to_update:
        new_slug = slugify(cat_name)
        
        cursor.execute(
            "UPDATE categories SET slug = %s WHERE id = %s",
            (new_slug, cat_id)
        )
        print(f"Updated category ID {cat_id}: '{cat_name}' -> slug: '{new_slug}'")
    
    # Commit changes
    conn.commit()
    print(f"\nSuccessfully updated {len(categories_to_update)} category slugs")
    
except Exception as e:
    conn.rollback()
    print(f"Error: {e}")
    
finally:
    cursor.close()
    conn.close()

Updated category ID 1: 'Thời Trang' -> slug: 'thoi-trang'
Updated category ID 2: 'Điện Tử' -> slug: 'dien-tu'
Updated category ID 3: 'Nghệ Thuật & Trang Sức' -> slug: 'nghe-thuat-trang-suc'
Updated category ID 4: 'Thời Trang Nam' -> slug: 'thoi-trang-nam'
Updated category ID 5: 'Thời Trang Nữ' -> slug: 'thoi-trang-nu'
Updated category ID 6: 'Giày Dép' -> slug: 'giay-dep'
Updated category ID 7: 'Điện Thoại & Máy Tính Bảng' -> slug: 'dien-thoai-may-tinh-bang'
Updated category ID 8: 'Laptop & PC' -> slug: 'laptop-pc'
Updated category ID 9: 'Phụ Kiện Số' -> slug: 'phu-kien-so'
Updated category ID 10: 'Máy Ảnh & Quay Phim' -> slug: 'may-anh-quay-phim'
Updated category ID 11: 'Tranh Ảnh' -> slug: 'tranh-anh'
Updated category ID 12: 'Tượng & Điêu Khắc' -> slug: 'tuong-dieu-khac'
Updated category ID 13: 'Trang Sức' -> slug: 'trang-suc'

Successfully updated 13 category slugs


## 4. Execute SQL Statements to Supabase